In [23]:
import time
import re
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver

# Liste des liens pour chaque catégorie


categories = [
    #"toutes les offres d'emploi",
    "administrations,moyens généraux",
   "assistanat,secrétariat",
    "comptabilite,finance",
   "metiers banque et assurances",
   "juridique,fiscal,audit,conseil",
    "RH,personnel,formation",
    "education,enseignement",
   "direction générale,direction d'unité",
    "vente,televente,assistanat",
    "commercial,technico commercial,service client",
    "responsable commercial,grands comptes",
    "créatio, design",
    "marketing, communication",
    "journalisme,médias,traduction",
    "informatique,systèmes d'information,internet",          
    "télécommunication,réseaux",
    "chantier,métiers BTP,architecture",
   "ingénierie,etudes,projet,R&D",
    "logistique,achat,stock,transport",
    "production,méthode,industrie",
    "maintenance,entretien",
    "Qualité,sécurité,Environnement",
    "Santé,Médical,Pharmacie",
    "Hotellerie,Tourisme,Restauration, Loisirs",
    "Ouvriers qualifiés, Chauffeur",
    "autre",              
    "Métiers de l'agriculture" 
]
base_url = "https://www.novojob.com/cote-d-ivoire/offres-d-emploi?q="
category_links = [f"{base_url}{'+'.join(category.split(','))}" for category in categories]

intitules_list = []
entreprises_list = []
pays_list = []
dates_list = []
lien_list = []
niveau_list = []
experience_list = []

# Utilisation d'un en-tête pour éviter d'être bloqué
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

# Parcourir les liens de chaque catégorie
for category_link in category_links:
    req = requests.get(category_link, headers=headers)
    soup = BeautifulSoup(req.text, 'html.parser')
    time.sleep(5)  # Attendre 5 secondes avant la prochaine requête

    if 'finance' in category_link:
        offres = soup.find_all('h2', class_='ellipsis row-fluid')
        entreprises = soup.find_all('h6', class_='ellipsis')
        niveaux = soup.find_all('span', class_='spaced-right phone-display-blok')
    else:
        offres = soup.find_all('h2', class_='ellipsis row-fluid')
        entreprises = soup.find_all('h6', class_='ellipsis')
        niveaux = soup.find_all('span', class_='spaced-right phone-display-blok')

    for offre, entreprise, niveau in zip(offres, entreprises, niveaux):
        bloc_bottom = offre.find_next('div', class_='bloc-bottom')
        intitules_list.append(offre.get_text().strip())
        entreprises_list.append(entreprise.get_text().strip())
        lien_list.append(category_links.index(category_link))

        # Les informations (pays, date, niveau, expérience) sont contenues dans la même span, nous devons les séparer
        pays_info = bloc_bottom.find('i', class_='fa fa-map-marker icon-left')
        pays = pays_info.find_parent().text.strip() if pays_info else None
        pays_list.append(pays)

        date_info = bloc_bottom.find('i', class_='fa fa-clock-o icon-left')
        date = date_info.find_parent().text.strip() if date_info else None
        dates_list.append(date)

        # Ajout des colonnes pour le niveau du poste et l'expérience demandée
        niveau_info = niveau.find('i', class_='fa fa-bookmark icon-left')
        niveau_text = niveau_info.find_parent().text.strip() if niveau_info else None

        # Utiliser une expression régulière pour extraire les informations de niveau et d'expérience
        match = re.match(r'(.+) \((.+)\)', niveau_text)

        if match:
            niveau_col, experience_col = match.groups()
        else:
            niveau_col, experience_col = None, None

        niveau_list.append(niveau_col)
        experience_list.append(experience_col)


# En-tête pour éviter d'être bloqué
en_tete = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

# Initialiser le pilote Selenium
driver = webdriver.Chrome()

# Liste pour stocker les détails de chaque emploi
all_job_details = []

# Parcourir les liens de chaque catégorie
for category_link in category_links:
    req = requests.get(category_link, headers=headers)
    soup = BeautifulSoup(req.text, 'html.parser')
    time.sleep(5)  # Attendre 5 secondes avant la prochaine requête

    offres = soup.find_all('div', class_='row-fluid job-details pointer')

    # Parcourir les offres d'emploi sur la page principale
    for offre in offres:
        # Trouver la balise <a> dans la structure HTML pour extraire le lien
        offre_link_tag = offre.find('a')

        # Vérifier si la balise <a> a été trouvée
        if offre_link_tag:
            # Extraire le lien de l'attribut 'href'
            offre_link = offre_link_tag['href']

            # Accéder à la page de l'offre pour collecter plus d'informations
            driver.get(offre_link)
            time.sleep(200)  # Attendre un peu pour que la page se charge après le clic

            # Récupérer le contenu de la page après le clic
            soup_offre = BeautifulSoup(driver.page_source, 'html.parser')

            # Nouveau dictionnaire pour stocker les détails de l'offre
            job_details = {}

            # Extracting job details
            details_section = soup_offre.find('ul', class_='text-small')
            if details_section:
                for li in details_section.find_all('li', class_='row-fluid'):
                    key = li.find('span', class_='span4').text.strip()
                    value = li.find('span', class_='span8').text.strip()
                    job_details[key] = value

            # Extracting the provided text
            description_section = soup_offre.find('div', class_='spaced details-description')
            if description_section:
                provided_text = description_section.text.strip()
                job_details['Provided Text'] = provided_text

            # Ajouter les détails de l'emploi à la liste
            all_job_details.append(job_details)

# Fermer le pilote Selenium à la fin
driver.quit()

# Convertir les détails des offres d'emploi en DataFrame
df_Novojob = pd.DataFrame(all_job_details)

df_Novojob


WebDriverException: Message: disconnected: not connected to DevTools
  (failed to check if window was closed: disconnected: not connected to DevTools)
  (Session info: chrome=120.0.6099.225)
Stacktrace:
	GetHandleVerifier [0x00007FF73E7B2142+3514994]
	(No symbol) [0x00007FF73E3D0CE2]
	(No symbol) [0x00007FF73E2776AA]
	(No symbol) [0x00007FF73E25FD6A]
	(No symbol) [0x00007FF73E25FC20]
	(No symbol) [0x00007FF73E2799A1]
	(No symbol) [0x00007FF73E3021B7]
	(No symbol) [0x00007FF73E2E5D93]
	(No symbol) [0x00007FF73E2B4BDC]
	(No symbol) [0x00007FF73E2B5C64]
	GetHandleVerifier [0x00007FF73E7DE16B+3695259]
	GetHandleVerifier [0x00007FF73E836737+4057191]
	GetHandleVerifier [0x00007FF73E82E4E3+4023827]
	GetHandleVerifier [0x00007FF73E5004F9+689705]
	(No symbol) [0x00007FF73E3DC048]
	(No symbol) [0x00007FF73E3D8044]
	(No symbol) [0x00007FF73E3D81C9]
	(No symbol) [0x00007FF73E3C88C4]
	BaseThreadInitThunk [0x00007FF9229B257D+29]
	RtlUserThreadStart [0x00007FF92370AA58+40]


In [ ]:
df_offers = pd.DataFrame({
    'Intitule': intitules_list,
    'Entreprise': entreprises_list,
    'Pays': pays_list,
    'Date': dates_list,
    'Niveau': niveau_list,
    'Experience_lettre': experience_list,
    'Lien': lien_list
})
df_offers

In [ ]:
# Ajouter les listes existantes en tant que colonnes au DataFrame
df_Novojob['Intitule'] = intitules_list
df_Novojob['Entreprise'] = entreprises_list
df_Novojob['Pays'] = pays_list
df_Novojob['Date'] = dates_list
df_Novojob['Niveau'] = niveau_list
df_Novojob['Experience_lettre'] = experience_list
df_Novojob['Lien'] = lien_list


In [ ]:
# Réorganiser les colonnes selon vos besoins
df_Novojob = df[[
    'Intitule', 'Entreprise', 'Pays', 'Date', 'Niveau', 'Experience_lettre',
    'Lien', 'Lieu de travail', "Date d'expiration", 'Niveau de poste', "Secteur d'activité", "Niveau d'étude (diplome)",
    "Nombre de postes", "Type de contrat", "Provided Text"
]]

# Afficher le DataFrame
df_Novojob


In [3]:
df_Novojob = pd.DataFrame(all_job_details)
df_Novojob

,Lieu de travail,Date d'expiration,Niveau de poste,Secteur d'activité,Niveau d'étude (diplome),Nombre de postes,Type de contrat,Provided Text,Nom de l'entreprise
0,"Abidjan, Côte d'ivoire",04 Février,Manager / Responsable département,Services,"Master 1, Licence Bac + 4| Master 2, Ingénior...",01,Consultant,Notre client une société qui vient de se const...,NaN
1,"Abidjan, Côte d'ivoire",15 Février,Confirmé / Expérimenté,"Banque, Assurance, Finance","Master 2, Ingéniorat, Bac + 5",02,CDI,Missions du posteSous la supervision du Respon...,NaN
2,Côte d'ivoire,Aujourd'hui,Confirmé / Expérimenté,Industries,"Master 1, Licence Bac + 4| Master 2, Ingénior...",01 poste ouvert,CDI,ENTITE : ...,Société Générale Côte D'ivoire
3,Côte d'ivoire,28 Février,Confirmé / Expérimenté,Industries,"Master 2, Ingéniorat, Bac + 5",01 poste ouvert,CDI| CDD,ENTITE : ...,Société Générale Côte D'ivoire
4,"Abidjan, Côte d'ivoire",21 Février,Confirmé / Expérimenté,"Banque, Assurance, Finance","Master 2, Ingéniorat, Bac + 5",01,CDI,AVIS DE RECRUTEMENT FAMILLE D'EMPLOIS : DEVELO...,NaN
...,...,...,...,...,...,...,...,...,...
500,"Abidjan, Côte d'ivoire",03 Mars,Débutant / Junior,"Banque, Assurance, Finance",NaN,01,CDI| CDD,La banque en toute confiance. Acteur majeur d...,NaN
501,"Abidjan, Côte d'ivoire",07 Février,Responsable d'équipe| Confirmé / Expérimenté,"Banque, Assurance, Finance",NaN,01,CDI,Nous recrutons pour l'Inspection Générale de l...,NaN
502,"Abidjan, Côte d'ivoire",22 Février,Confirmé / Expérimenté,"Informatique, Télécom, Internet","Master 2, Ingéniorat, Bac + 5",01,CDI,ASCENS SERVICES est une filiale du Groupe Afri...,NaN
503,Côte d'ivoire,28 Février,Confirmé / Expérimenté,Industries,"Master 2, Ingéniorat, Bac + 5",01 poste ouvert,CDD,VOS MISSIONS AU QUOTIDIENSous l'autorité du Ch...,Société Générale Côte D'ivoire


In [ ]:
categories = [
    #"toutes les offres d'emploi",
    "administrations,moyens généraux",
   "assistanat,secrétariat",
    "comptabilite,finance",
   "metiers banque et assurances",
   "juridique,fiscal,audit,conseil",
    "RH,personnel,formation",
    "education,enseignement",
   "direction générale,direction d'unité",
    "vente,televente,assistanat",
    "commercial,technico commercial,service client",
    "responsable commercial,grands comptes",
    "créatio, design",
    "marketing, communication",
    "journalisme,médias,traduction",
    "informatique,systèmes d'information,internet",          
    "télécommunication,réseaux",
    "chantier,métiers BTP,architecture",
   "ingénierie,etudes,projet,R&D",
    "logistique,achat,stock,transport",
    "production,méthode,industrie",
    "maintenance,entretien",
    "Qualité,sécurité,Environnement",
    "Santé,Médical,Pharmacie",
    "Hotellerie,Tourisme,Restauration, Loisirs",
    "Ouvriers qualifiés, Chauffeur",
    "autre",              
    "Métiers de l'agriculture" 
]

base_url = "https://www.novojob.com/cote-d-ivoire/offres-d-emploi?q="
category_links = [f"{base_url}{'+'.join(category.split(','))}" for category in categories]

In [16]:
import time
import re
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver

# Liste des liens pour chaque catégorie
categories = [
    #"toutes les offres d'emploi",
    "administrations,moyens généraux",
   "assistanat,secrétariat",
    "comptabilite,finance",
   "metiers banque et assurances",
   "juridique,fiscal,audit,conseil",
    "RH,personnel,formation",
    "education,enseignement",
   "direction générale,direction d'unité",
    "vente,televente,assistanat",
    "commercial,technico commercial,service client",
    "responsable commercial,grands comptes",
    "créatio, design",
    "marketing, communication",
    "journalisme,médias,traduction",
    "informatique,systèmes d'information,internet",          
    "télécommunication,réseaux",
    "chantier,métiers BTP,architecture",
   "ingénierie,etudes,projet,R&D",
    "logistique,achat,stock,transport",
    "production,méthode,industrie",
    "maintenance,entretien",
    "Qualité,sécurité,Environnement",
    "Santé,Médical,Pharmacie",
    "Hotellerie,Tourisme,Restauration, Loisirs",
    "Ouvriers qualifiés, Chauffeur",
    "autre",              
    "Métiers de l'agriculture" 
]

base_url = "https://www.novojob.com/cote-d-ivoire/offres-d-emploi?q="
category_links = [f"{base_url}{'+'.join(category.split(','))}" for category in categories]
intitules_list = []
entreprises_list = []
pays_list = []
dates_list = []
lien_list = []
niveau_list = []
experience_list = []

# Utilisation d'un en-tête pour éviter d'être bloqué
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

# En-tête pour éviter d'être bloqué
en_tete = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

# Initialiser le pilote Selenium
driver = webdriver.Chrome()

# Liste pour stocker les détails de chaque emploi
all_job_details = []


# Parcourir les liens de chaque catégorie
for category_link in category_links:
    req = requests.get(category_link, headers=headers)
    soup = BeautifulSoup(req.text, 'html.parser')
    time.sleep(5)  # Attendre 5 secondes avant la prochaine requête

    if 'finance' in category_link:
        offres = soup.find_all('h2', class_='ellipsis row-fluid')
        entreprises = soup.find_all('h6', class_='ellipsis')
        niveaux = soup.find_all('span', class_='spaced-right phone-display-blok')
    else:
        offres = soup.find_all('h2', class_='ellipsis row-fluid')
        entreprises = soup.find_all('h6', class_='ellipsis')
        niveaux = soup.find_all('span', class_='spaced-right phone-display-blok')

    for offre, entreprise, niveau in zip(offres, entreprises, niveaux):
        bloc_bottom = offre.find_next('div', class_='bloc-bottom')
        intitules_list.append(offre.get_text().strip())
        entreprises_list.append(entreprise.get_text().strip())
        lien_list.append(category_links.index(category_link))

        # Les informations (pays, date, niveau, expérience) sont contenues dans la même span, nous devons les séparer
        pays_info = bloc_bottom.find('i', class_='fa fa-map-marker icon-left')
        pays = pays_info.find_parent().text.strip() if pays_info else None
        pays_list.append(pays)

        date_info = bloc_bottom.find('i', class_='fa fa-clock-o icon-left')
        date = date_info.find_parent().text.strip() if date_info else None
        dates_list.append(date)

        # Ajout des colonnes pour le niveau du poste et l'expérience demandée
        niveau_info = niveau.find('i', class_='fa fa-bookmark icon-left')
        niveau_text = niveau_info.find_parent().text.strip() if niveau_info else None

        # Utiliser une expression régulière pour extraire les informations de niveau et d'expérience
        match = re.match(r'(.+) \((.+)\)', niveau_text)

        if match:
            niveau_col, experience_col = match.groups()
        else:
            niveau_col, experience_col = None, None

        niveau_list.append(niveau_col)
        experience_list.append(experience_col)

    offres = soup.find_all('div', class_='row-fluid job-details pointer')

    # Parcourir les offres d'emploi sur la page principale
    for offre in offres:
        # Trouver la balise <a> dans la structure HTML pour extraire le lien
        offre_link_tag = offre.find('a')

        # Vérifier si la balise <a> a été trouvée
        if offre_link_tag:
            # Extraire le lien de l'attribut 'href'
            offre_link = offre_link_tag['href']

            # Accéder à la page de l'offre pour collecter plus d'informations
            driver.get(offre_link)
            time.sleep(20)  # Attendre un peu pour que la page se charge après le clic

            # Récupérer le contenu de la page après le clic
            soup_offre = BeautifulSoup(driver.page_source, 'html.parser')

            # Nouveau dictionnaire pour stocker les détails de l'offre
            job_details = {}

            # Extracting job details
            details_section = soup_offre.find('ul', class_='text-small')
            if details_section:
                for li in details_section.find_all('li', class_='row-fluid'):
                    key = li.find('span', class_='span4').text.strip()
                    value = li.find('span', class_='span8').text.strip()
                    job_details[key] = value

            # Extracting the provided text
            description_section = soup_offre.find('div', class_='spaced details-description')
            if description_section:
                provided_text = description_section.text.strip()
                job_details['Provided Text'] = provided_text

            # Ajouter les détails de l'emploi à la liste
            all_job_details.append(job_details)

# Fermer le pilote Selenium à la fin
driver.quit()
# Convertir les détails des offres d'emploi en DataFrame






WebDriverException: Message: disconnected: not connected to DevTools
  (failed to check if window was closed: disconnected: not connected to DevTools)
  (Session info: chrome=120.0.6099.109)
Stacktrace:
	GetHandleVerifier [0x00007FF73E7B2142+3514994]
	(No symbol) [0x00007FF73E3D0CE2]
	(No symbol) [0x00007FF73E2776AA]
	(No symbol) [0x00007FF73E25FD6A]
	(No symbol) [0x00007FF73E25FC20]
	(No symbol) [0x00007FF73E2799A1]
	(No symbol) [0x00007FF73E3021B7]
	(No symbol) [0x00007FF73E2E5D93]
	(No symbol) [0x00007FF73E2B4BDC]
	(No symbol) [0x00007FF73E2B5C64]
	GetHandleVerifier [0x00007FF73E7DE16B+3695259]
	GetHandleVerifier [0x00007FF73E836737+4057191]
	GetHandleVerifier [0x00007FF73E82E4E3+4023827]
	GetHandleVerifier [0x00007FF73E5004F9+689705]
	(No symbol) [0x00007FF73E3DC048]
	(No symbol) [0x00007FF73E3D8044]
	(No symbol) [0x00007FF73E3D81C9]
	(No symbol) [0x00007FF73E3C88C4]
	BaseThreadInitThunk [0x00007FF9229B257D+29]
	RtlUserThreadStart [0x00007FF92370AA58+40]


In [17]:
df_Novojob = pd.DataFrame(all_job_details)
df_Novojob

,Lieu de travail,Date d'expiration,Niveau de poste,Secteur d'activité,Niveau d'étude (diplome),Nombre de postes,Type de contrat,Provided Text,Nom de l'entreprise
0,"Abidjan, Côte d'ivoire",04 Février,Manager / Responsable département,Services,"Master 1, Licence Bac + 4| Master 2, Ingénior...",01,Consultant,Notre client une société qui vient de se const...,NaN
1,"Abidjan, Côte d'ivoire",15 Février,Confirmé / Expérimenté,"Banque, Assurance, Finance","Master 2, Ingéniorat, Bac + 5",02,CDI,Missions du posteSous la supervision du Respon...,NaN
2,Côte d'ivoire,Aujourd'hui,Confirmé / Expérimenté,Industries,"Master 1, Licence Bac + 4| Master 2, Ingénior...",01 poste ouvert,CDI,ENTITE : ...,Société Générale Côte D'ivoire
3,Côte d'ivoire,28 Février,Confirmé / Expérimenté,Industries,"Master 2, Ingéniorat, Bac + 5",01 poste ouvert,CDI| CDD,ENTITE : ...,Société Générale Côte D'ivoire
4,"Abidjan, Côte d'ivoire",21 Février,Confirmé / Expérimenté,"Banque, Assurance, Finance","Master 2, Ingéniorat, Bac + 5",01,CDI,AVIS DE RECRUTEMENT FAMILLE D'EMPLOIS : DEVELO...,NaN
...,...,...,...,...,...,...,...,...,...
128,Côte d'ivoire,04 Février,Débutant / Junior,"Informatique, Télécom, Internet","Master 1, Licence Bac + 4",01,CDD,Le Conseil des Institutions Théologiques d’Afr...,NaN
129,Côte d'ivoire,28 Février,Confirmé / Expérimenté,Industries,"Master 2, Ingéniorat, Bac + 5",01 poste ouvert,CDI| CDD,ENTITE : ...,Société Générale Côte D'ivoire
130,"Abidjan, Côte d'ivoire",04 Février,Responsable d'équipe| Confirmé / Expérimenté,Services,"Licence (LMD), Bac + 3| Master 1, Licence Bac...",01,CDI| CDD,Le FOOD COURT est un restaurant qui permettra ...,NaN
131,"Abidjan, Côte d'ivoire",04 Février,Manager / Responsable département,Services,"Master 1, Licence Bac + 4| Master 2, Ingénior...",01,Consultant,Notre client une société qui vient de se const...,NaN


In [ ]:
# Ajouter les listes existantes en tant que colonnes au DataFrame
df_Novojob['Intitule'] = intitules_list
df_Novojob['Entreprise'] = entreprises_list
df_Novojob['Pays'] = pays_list
df_Novojob['Date'] = dates_list
df_Novojob['Niveau'] = niveau_list
df_Novojob['Experience_lettre'] = experience_list
df_Novojob['Lien'] = lien_list


In [ ]:
# Réorganiser les colonnes selon vos besoins
df_Novojob = df_Novojob[[
    'Intitule', 'Entreprise', 'Pays', 'Date', 'Niveau', 'Experience_lettre',
    'Lien', 'Lieu de travail', "Date d'expiration", 'Niveau de poste', "Secteur d'activité", "Niveau d'étude (diplome)",
    "Nombre de postes", "Type de contrat", "Provided Text"
]]

# Afficher le DataFrame
df_Novojob
